# How to get the data from the Brazilian 2018 Elections
* Target url = http://www.tse.jus.br/hotsites/pesquisas-eleitorais/prestacao_contas_anos/2018.html

## 1. Import libraries

In [1]:
import os
import io
import requests
import numpy as np
import pandas as pd
from zipfile import ZipFile
from datetime import datetime
from bs4 import BeautifulSoup as bs

## 2. Scrap the url for the source zip files and download it

In [2]:
target_url  = r'http://www.tse.jus.br/hotsites/pesquisas-eleitorais/prestacao_contas_anos/2018.html'  # for revenues and expenses data
target_url2 = r'http://www.tse.jus.br/hotsites/pesquisas-eleitorais/candidatos_anos/2018.html'        # fort assets and candidates data

r           = requests.get(target_url, timeout=3) # do http request to get raw html
r2          = requests.get(target_url2, timeout=3)

soup        = bs(r.text)                          # create BeautifulSoup object to scrap tags with the zip source files url
soup2       = bs(r2.text)

zips_url    = [a.attrs['href'] for a in soup.select('p a')]
zips_url2   = [a.attrs['href'] for a in soup2.select('p a')]

zips_url + zips_url2

['http://agencia.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_de_contas_eleitorais_orgaos_partidarios_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_de_contas_eleitorais_candidatos_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/prestacao_contas/CNPJ_campanha_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_coligacao/consulta_coligacao_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_vagas/consulta_vagas_2018.zip',
 'http://agencia.tse.jus.br/estatistica/sead/odsele/motivo_cassacao/motivo_cassacao_2018.zip']

In [4]:
cand_urls = [x for x in (zips_url + zips_url2) if 'cand' in x] # filter just urls related to candidates data

zip_folder = './src_zip'

os.makedirs(zip_folder, exist_ok=True)          # check if there is a folder to download zip files; create one if there is not.

for url in cand_urls:                            # start downloading each zip file
    r = requests.get(url)
    z = ZipFile(io.BytesIO(r.content))
    z.extractall(zip_folder)

In [13]:
for file in os.listdir(zip_folder): # delete all downloaded files that does not have the whole Brazil data
    if '2018_BRASIL' not in file:  
        os.remove(os.path.join(zip_folder, file))

In [4]:
os.listdir(zip_folder)

['bem_candidato_2018_BRASIL.csv',
 'consulta_cand_2018_BRASIL.csv',
 'despesas_contratadas_candidatos_2018_BRASIL.csv',
 'despesas_pagas_candidatos_2018_BRASIL.csv',
 'receitas_candidatos_2018_BRASIL.csv',
 'receitas_candidatos_doador_originario_2018_BRASIL.csv']

## 3. Scan downloaded files

In [5]:
a, b, c, d, e, f = [os.path.join(zip_folder, j) for j in os.listdir(zip_folder)]
dfa, dfb, dfc, dfd, dfe, dff = [pd.read_csv(j, sep=';', encoding='Latin1', decimal=',') for j in [a, b, c, d, e, f]]

In [6]:
dfa.head(2) # assets own by a candidate

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO
0,28/02/2020,19:10:28,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,RR,RR,RORAIMA,230000615935,3,99,OUTROS BENS E DIREITOS,50 cabeça de gado,130000.0,12/10/2018,18:08:08
1,28/02/2020,19:10:28,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,RJ,RJ,RIO DE JANEIRO,190000616260,1,12,Casa,Bairro Parque Equitativa - Duque de Caxias - RJ,15000.0,30/04/2019,15:26:08


In [7]:
dfb.head(2) # candidate metadata

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO
0,28/02/2020,19:10:49,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,BRANCA,257,EMPRESÁRIO,0,5,SUPLENTE,N,S,-1,6007507020186160000
1,28/02/2020,19:10:49,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,131,ADVOGADO,0,5,SUPLENTE,N,S,-1,6035741820186190000


In [8]:
dfc.head(2) # hired expenses

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,SG_PARTIDO_FORNECEDOR,NM_PARTIDO_FORNECEDOR,DS_TIPO_DOCUMENTO,NR_DOCUMENTO,CD_ORIGEM_DESPESA,DS_ORIGEM_DESPESA,SQ_DESPESA,DT_DESPESA,DS_DESPESA,VR_DESPESA_CONTRATADA
0,27/02/2020,18:57:37,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,#NULO#,#NULO#,#NULO#,#NULO#,20210000,"Encargos financeiros, taxas bancárias e/ou op....",24845061,21/09/2018,TARIFA TRANSF. REC. E/I REF. A 21/09,1.15
1,27/02/2020,18:57:37,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,#NULO#,#NULO#,#NULO#,#NULO#,20210000,"Encargos financeiros, taxas bancárias e/ou op....",24845063,27/09/2018,TAR. TRANSF. RECURSO E/I,1.15


In [9]:
dfd.head(2) # expenses settled

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,DS_ORIGEM_DESPESA,CD_NATUREZA_DESPESA,DS_NATUREZA_DESPESA,CD_ESPECIE_RECURSO,DS_ESPECIE_RECURSO,SQ_DESPESA,SQ_PARCELAMENTO_DESPESA,DT_PAGTO_DESPESA,DS_DESPESA,VR_PAGTO_DESPESA
0,27/02/2020,17:36:46,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,Despesas com pessoal,1,Financeiro,1,Financeiro,25062587,17267523,28/09/2018,Despesas com pessoal,540.6
1,27/02/2020,17:36:46,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,Despesas com pessoal,1,Financeiro,1,Financeiro,25062623,17267538,28/09/2018,Despesas com pessoal,540.6


In [10]:
dfe.head(2) # candidate revenues

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,DS_CARGO_CANDIDATO_DOADOR,NR_PARTIDO_DOADOR,SG_PARTIDO_DOADOR,NM_PARTIDO_DOADOR,NR_RECIBO_DOACAO,NR_DOCUMENTO_DOACAO,SQ_RECEITA,DT_RECEITA,DS_RECEITA,VR_RECEITA
0,27/02/2020,19:15:01,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Parcial,...,#NULO#,44,PRP,Partido Republicano Progressista,044120600000RS000001E,A35F311441725745030,10670220,31/08/2018,#NULO#,2500.00
1,27/02/2020,19:15:01,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Parcial,...,Governador,18,REDE,Rede Sustentabilidade,#NULO#,#NULO#,10673281,30/08/2018,SANTINHOS,54.86


In [11]:
dff.head(2) # candidates revenues with tracking of the original source of the donation, but few records

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,NR_CPF_CNPJ_DOADOR_ORIGINARIO,NM_DOADOR_ORIGINARIO,NM_DOADOR_ORIGINARIO_RFB,TP_DOADOR_ORIGINARIO,CD_CNAE_DOADOR_ORIGINARIO,DS_CNAE_DOADOR_ORIGINARIO,SQ_RECEITA,DT_RECEITA,DS_RECEITA,VR_RECEITA
0,27/02/2020,17:09:10,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,99761157920,Aleta dreves,ALETA TEREZA DREVES,F,-1,#NULO#,12067691,04/09/2018,Recursos de Financiamento Coletivo,50.0
1,27/02/2020,17:09:10,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,689115210,Andreska Alves de Oliveira Abejdid,ANDRESKA ALVES DE OLIVEIRA ABEJDID,F,-1,#NULO#,12067691,31/08/2018,Recursos de Financiamento Coletivo,50.0


## 4. Clean the data

In [14]:
# CLEAN ASSETS DATA

assets_columns = ['SQ_CANDIDATO',            # PRIMARY KEY to track candidate id
                  'DS_TIPO_BEM_CANDIDATO',   # type of asset
                  'DS_BEM_CANDIDATO',        # details of the asset
                  'VR_BEM_CANDIDATO',        # asset net value
                  'DT_ULTIMA_ATUALIZACAO']   # when this asset was informed

# filter columns
assets                     = dfa[assets_columns]

# clean null values on asset description / details
assets.loc[ : ,'DS_BEM_CANDIDATO'] = assets.loc[ : ,'DS_BEM_CANDIDATO'].map({'#NULO#': 'NOT DETAILED'}).fillna('NOT DETAILED')

# format BRZ date format to pandas datetime
assets.loc[ : ,'DT_ULTIMA_ATUALIZACAO'] = pd.to_datetime(assets.loc[ : ,'DT_ULTIMA_ATUALIZACAO'], format='%d/%m/%Y')

assets.head(2)

,SQ_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO
0,230000615935,OUTROS BENS E DIREITOS,NOT DETAILED,130000.0,2018-10-12
1,190000616260,Casa,NOT DETAILED,15000.0,2019-04-30


In [44]:
# CLEAN CANDIDATE METADATA
# REMOVE ALL COLUMNS WITH SOME SORT OF CODE OTHER THAN SQ_CANDIDATO
cols = [i for i in dfb.columns if 'CD_' not in i]

# scan columns and select columns to remove in the below filter
#dfb[cols].head(3).T.sort_index()

rm_cols = ['HH_GERACAO', 'DT_GERACAO', 'NM_SOCIAL_CANDIDATO', 'NR_DESPESA_MAX_CAMPANHA', 'NR_PROCESSO', 'NR_PROTOCOLO_CANDIDATURA', 
           'NR_TITULO_ELEITORAL_CANDIDATO', 'SQ_COLIGACAO', 'SG_UE', 'NM_TIPO_ELEICAO', 'TP_ABRANGENCIA']

In [58]:
aux_df = dfb[cols]
cand   = aux_df[list(set(aux_df.columns).difference(set(rm_cols)))]
cand.loc[ : , 'DT_ELEICAO'] = pd.to_datetime(cand.loc[ : , 'DT_ELEICAO'], format='%d/%m/%Y')
#cand.loc[ : , 'DT_NASCIMENTO'] = pd.to_datetime(cand.loc[ : , 'DT_NASCIMENTO'], format='%d/%m/%Y')
cand.head(2)

,NR_PARTIDO,SG_PARTIDO,NR_CANDIDATO,DS_NACIONALIDADE,DS_CARGO,NR_IDADE_DATA_POSSE,NM_COLIGACAO,ST_REELEICAO,NR_TURNO,DS_COR_RACA,...,DS_ESTADO_CIVIL,NM_MUNICIPIO_NASCIMENTO,SG_UF,NM_URNA_CANDIDATO,NM_EMAIL,DS_GRAU_INSTRUCAO,DS_OCUPACAO,NR_CPF_CANDIDATO,NM_UE,DS_ELEICAO
0,54,PPL,54007,BRASILEIRA NATA,DEPUTADO ESTADUAL,38,Coligação do bem e da verdade para mudar o Paraná,N,1,BRANCA,...,SOLTEIRO(A),CURITIBA,PR,BIBERSSON CESAR-CIDADÃO FISCAL,BIBERSONCESAR@GMAIL.COM,SUPERIOR INCOMPLETO,EMPRESÁRIO,2610091923,PARANÁ,Eleições Gerais Estaduais 2018
1,44,PRP,4419,BRASILEIRA NATA,DEPUTADO FEDERAL,74,PARTIDO ISOLADO,N,1,PARDA,...,CASADO(A),SÃO PAULO,RJ,DOMINGOS FREITAS,DOMINGOSPPFREITAS@GMAIL.COM,SUPERIOR COMPLETO,ADVOGADO,10693360763,RIO DE JANEIRO,Eleições Gerais Estaduais 2018


In [74]:
# CLEAN HIRED EXPENSES
cols = ['SQ_CANDIDATO', 'SQ_PRESTADOR_CONTAS', 'DS_DESPESA', 'DT_DESPESA', 'DT_PRESTACAO_CONTAS', 'NM_FORNECEDOR', 'NM_FORNECEDOR_RFB', 'NR_CNPJ_PRESTADOR_CONTA', 'TP_PRESTACAO_CONTAS', 'VR_DESPESA_CONTRATADA']
hired_ex = dfc[cols]
hired_ex.loc[ : , 'DT_DESPESA'] = pd.to_datetime(hired_ex.loc[ : , 'DT_DESPESA'], format='%d/%m/%Y')
hired_ex.loc[ : , 'DT_PRESTACAO_CONTAS'] = pd.to_datetime(hired_ex.loc[ : , 'DT_PRESTACAO_CONTAS'], format='%d/%m/%Y')
hired_ex.head(2)

,SQ_CANDIDATO,SQ_PRESTADOR_CONTAS,DS_DESPESA,DT_DESPESA,DT_PRESTACAO_CONTAS,NM_FORNECEDOR,NM_FORNECEDOR_RFB,NR_CNPJ_PRESTADOR_CONTA,TP_PRESTACAO_CONTAS,VR_DESPESA_CONTRATADA
0,10000608399,420242136,TARIFA TRANSF. REC. E/I REF. A 21/09,2018-09-21,2019-01-22,#NULO#,#NULO#,31194592000153,Final,1.15
1,10000608399,420242136,TAR. TRANSF. RECURSO E/I,2018-09-27,2019-01-22,#NULO#,#NULO#,31194592000153,Final,1.15


In [78]:
# CLEAN SETTLED EXPENSES
cols = ['SQ_PRESTADOR_CONTAS', 'DS_NATUREZA_DESPESA', 'DS_FONTE_DESPESA', 'DS_ORIGEM_DESPESA', 'DT_PAGTO_DESPESA', 'DT_PRESTACAO_CONTAS', 'TP_PRESTACAO_CONTAS', 'VR_PAGTO_DESPESA']
settl_ex = dfd[cols]
settl_ex.loc[ : , 'DT_PAGTO_DESPESA'] = pd.to_datetime(settl_ex.loc[ : , 'DT_PAGTO_DESPESA'], format='%d/%m/%Y')
settl_ex.loc[ : , 'DT_PRESTACAO_CONTAS'] = pd.to_datetime(settl_ex.loc[ : , 'DT_PRESTACAO_CONTAS'], format='%d/%m/%Y')
settl_ex.head(2)

,SQ_PRESTADOR_CONTAS,DS_NATUREZA_DESPESA,DS_FONTE_DESPESA,DS_ORIGEM_DESPESA,DT_PAGTO_DESPESA,DT_PRESTACAO_CONTAS,TP_PRESTACAO_CONTAS,VR_PAGTO_DESPESA
0,422812285,Financeiro,Outros Recursos,Despesas com pessoal,2018-09-28,2019-04-30,Final,540.6
1,422812285,Financeiro,Outros Recursos,Despesas com pessoal,2018-09-28,2019-04-30,Final,540.6


In [97]:
# CLEAN REVENUE DATA
cols = ['SQ_CANDIDATO', 'SQ_PRESTADOR_CONTAS', 'DS_ESPECIE_RECEITA', 'DS_NATUREZA_RECEITA', 'DS_ORIGEM_RECEITA', 'DS_RECEITA', 'DT_RECEITA', 'DT_PRESTACAO_CONTAS',
          'NM_DOADOR', 'NM_DOADOR_RFB','SG_PARTIDO_DOADOR', 'NM_PARTIDO_DOADOR', 'NR_CPF_CNPJ_DOADOR', 'ST_TURNO', 'TP_PRESTACAO_CONTAS', 'VR_RECEITA']
revenue = dfe[cols]
revenue.loc[ : , 'DT_RECEITA'] = pd.to_datetime(revenue.loc[ : , 'DT_RECEITA'], format='%d/%m/%Y')
revenue.loc[ : , 'DT_PRESTACAO_CONTAS'] = pd.to_datetime(revenue.loc[ : , 'DT_PRESTACAO_CONTAS'], format='%d/%m/%Y')
revenue.head(2)

,SQ_CANDIDATO,SQ_PRESTADOR_CONTAS,DS_ESPECIE_RECEITA,DS_NATUREZA_RECEITA,DS_ORIGEM_RECEITA,DS_RECEITA,DT_RECEITA,DT_PRESTACAO_CONTAS,NM_DOADOR,NM_DOADOR_RFB,SG_PARTIDO_DOADOR,NM_PARTIDO_DOADOR,NR_CPF_CNPJ_DOADOR,ST_TURNO,TP_PRESTACAO_CONTAS,VR_RECEITA
0,210000616466,422660745,Transferência eletrônica,Financeiro,Recursos de partido político,#NULO#,2018-08-31,2018-09-10,Direção Estadual/Distrital,PARTIDO REPUBLICANO PROGRESSISTA - PRP,PRP,Partido Republicano Progressista,11811475000184,1,Parcial,2500.00
1,110000622015,427242799,Estimado,Estimável,Recursos de outros candidatos,SANTINHOS,2018-08-30,2018-09-10,ELEICAO 2018 KELLEN ARTHUR PREZA NOGUEIRA,ELEICAO 2018 KELLEN ARTHUR PREZA NOGUEIRA GOVE...,REDE,Rede Sustentabilidade,31236523000165,1,Parcial,54.86


In [105]:
# CLEAN REVENUE DATA
dff.head(2).T.sort_index()
cols = ['SQ_PRESTADOR_CONTAS', 'NM_DOADOR_ORIGINARIO', 'NM_DOADOR_ORIGINARIO_RFB', 'DT_RECEITA', 'DT_PRESTACAO_CONTAS', 'TP_PRESTACAO_CONTAS', 'DS_RECEITA', 'VR_RECEITA']
other_rev = dff[cols]
other_rev.loc[ : , 'DT_RECEITA'] = pd.to_datetime(other_rev.loc[ : , 'DT_RECEITA'], format='%d/%m/%Y')
other_rev.loc[ : , 'DT_PRESTACAO_CONTAS'] = pd.to_datetime(other_rev.loc[ : , 'DT_PRESTACAO_CONTAS'], format='%d/%m/%Y')
other_rev.head(2)

,SQ_PRESTADOR_CONTAS,NM_DOADOR_ORIGINARIO,NM_DOADOR_ORIGINARIO_RFB,DT_RECEITA,DT_PRESTACAO_CONTAS,TP_PRESTACAO_CONTAS,DS_RECEITA,VR_RECEITA
0,399898761,Aleta dreves,ALETA TEREZA DREVES,2018-09-04,2018-11-06,Final,Recursos de Financiamento Coletivo,50.0
1,399898761,Andreska Alves de Oliveira Abejdid,ANDRESKA ALVES DE OLIVEIRA ABEJDID,2018-08-31,2018-11-06,Final,Recursos de Financiamento Coletivo,50.0


## 5. Backup and save physically cleaned data

In [110]:
assets.to_feather(os.path.join(zip_folder,'assets.ftr'))
cand.to_feather(os.path.join(zip_folder,'cand.ftr'))
hired_ex.to_feather(os.path.join(zip_folder,'hired_ex.ftr'))
settl_ex.to_feather(os.path.join(zip_folder,'settl_ex.ftr'))
revenue.to_feather(os.path.join(zip_folder,'revenue.ftr'))
other_rev.to_feather(os.path.join(zip_folder,'other_rev.ftr'))